In [1]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/dataset.zip", 'r')
zip_ref.extractall("/dataset")
zip_ref.close()

Mounted at /content/drive/


import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('../dataset/compressed videos')

label_types = os.listdir('../dataset/compressed videos')
print (label_types)  

In [ ]:
import numpy as np 
import pandas as pd 
import os



In [ ]:
import shutil
classes = os.listdir('../dataset/train')
for clas in classes :
  signs = os.listdir('../dataset/train' + '/'+clas)
  i = len(signs) //5 
  for sign in signs :
    if not os.path.exists( '../dataset/test/'+clas) :
      os.mkdir('../dataset/test/'+clas)
    shutil.move('../dataset/train' + '/'+clas +'/'+ sign, '../dataset/test/'+clas+'/'+sign)
    i -= 1 
    if i == 0 :
      break
    # shutil.move(old_path, new_path)


In [ ]:
dataset_path = os.listdir('/dataset/train')[:2]

signs = []

for item in dataset_path:
 # Get all the file names
 all_signs = os.listdir('../dataset/train' + '/' +item)

 # Add them to the list
 for sign in all_signs:
    signs.append((item, str('train' + '/' +item) + '/' + sign))
    
# Build a dataframe        
train_df = pd.DataFrame(data=signs, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

    tag               video_name
0  baby  train/baby/baby_291.mp4
1  baby  train/baby/baby_149.mp4
2  baby   train/baby/baby_51.mp4
3  baby  train/baby/baby_124.mp4
4  baby  train/baby/baby_389.mp4
    tag           video_name
683  me  train/me/me_364.mp4
684  me  train/me/me_363.mp4
685  me  train/me/me_314.mp4
686  me  train/me/me_328.mp4
687  me  train/me/me_410.mp4


In [ ]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [ ]:
dataset_path = os.listdir('../dataset/test')[:2]
print(dataset_path)


signs = []

for item in dataset_path:
 # Get all the file names
 all_signs = os.listdir('../dataset/test' + '/' +item)

 # Add them to the list
 for sign in all_signs:
    signs.append((item, str('test'+ '/' +item) + '/' + sign))
    
# Build a dataframe        
test_df = pd.DataFrame(data=signs, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['baby', 'me']
    tag              video_name
0  baby   test/baby/baby_15.mp4
1  baby  test/baby/baby_391.mp4
2  baby  test/baby/baby_388.mp4
3  baby  test/baby/baby_109.mp4
4  baby   test/baby/baby_84.mp4
    tag          video_name
167  me    test/me/me_9.mp4
168  me  test/me/me_299.mp4
169  me  test/me/me_200.mp4
170  me  test/me/me_331.mp4
171  me  test/me/me_114.mp4


In [ ]:
! pip install git+https://github.com/tensorflow/docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-ygj675m5
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-ygj675m5
  Resolved https://github.com/tensorflow/docs to commit 51cce0203d63843dc5c625928fbc824f0818e14a
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=180336 sha256=65b28dbf4ad48cf5697961a74fa957500b8926d7f9b25336a42d4b33f32222dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-hdziqd9h/wheels/3b/ee/a2/ab4d36a9a4af495bcb936f3e849d4b497b65fa40548a68d6c3
Successfully built tensorflow-docs


In [ ]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 688
Total videos for testing: 172


,Unnamed: 0,video_name,tag
34,34,train/baby/baby_186.mp4,baby
487,487,train/me/me_375.mp4,me
101,101,train/baby/baby_414.mp4,baby
635,635,train/me/me_102.mp4,me
622,622,train/me/me_399.mp4,me
565,565,train/me/me_312.mp4,me
30,30,train/baby/baby_34.mp4,baby
457,457,train/me/me_69.mp4,me
674,674,train/me/me_54.mp4,me
614,614,train/me/me_158.mp4,me


In [ ]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['baby', 'me']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 30

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048



In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "../dataset")
test_data, test_labels = prepare_all_videos(test_df, "../dataset")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 35ms/step
Frame features in train set: (688, 20, 2048)
Frame masks in train set: (688, 20)


In [ ]:
print(test_data)
print(train_data)

(array([[[0.2618834 , 0.37281495, 0.37541756, ..., 1.5595922 ,
         0.3697908 , 1.943027  ],
        [0.1695316 , 0.3059131 , 0.40439072, ..., 1.4002314 ,
         0.36123455, 1.8700073 ],
        [0.12583114, 0.32283616, 0.37979802, ..., 1.2118628 ,
         0.23661956, 1.8236144 ],
        ...,
        [0.3898633 , 0.9578274 , 0.43658993, ..., 1.351786  ,
         0.07593471, 0.6200685 ],
        [0.11897382, 0.630228  , 0.42818174, ..., 1.3746192 ,
         0.04588794, 0.39434367],
        [0.3449553 , 0.73448336, 0.23296838, ..., 1.3769946 ,
         0.03078718, 0.36271077]],

       [[1.0164671 , 0.94950897, 0.24077591, ..., 0.6977469 ,
         0.6260107 , 0.45684296],
        [1.0156806 , 0.9934008 , 0.18928601, ..., 0.8431274 ,
         0.6204099 , 0.42201364],
        [0.95905775, 1.0330533 , 0.22619465, ..., 0.9652083 ,
         0.70168793, 0.5115699 ],
        ...,
        [1.0776287 , 1.2739931 , 0.297186  , ..., 0.60952836,
         0.99254656, 0.82555467],
        [0.

In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 50
# Utility for running experiments.
def run_experiment():
    filepath = "tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.2,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/50
16/18 [=========================>....] - ETA: 0s - loss: 0.7015 - accuracy: 0.5781
Epoch 1: val_loss improved from inf to 0.82920, saving model to tmp/video_classifier
18/18 [==============================] - 13s 169ms/step - loss: 0.6968 - accuracy: 0.5836 - val_loss: 0.8292 - val_accuracy: 0.0000e+00
Epoch 2/50
17/18 [===========================>..] - ETA: 0s - loss: 0.6755 - accuracy: 0.6085
Epoch 2: val_loss did not improve from 0.82920
18/18 [==============================] - 0s 18ms/step - loss: 0.6759 - accuracy: 0.6036 - val_loss: 0.8471 - val_accuracy: 0.0000e+00
Epoch 3/50
16/18 [=========================>....] - ETA: 0s - loss: 0.6671 - accuracy: 0.6152
Epoch 3: val_loss did not improve from 0.82920
18/18 [==============================] - 0s 20ms/step - loss: 0.6665 - accuracy: 0.6182 - val_loss: 0.8913 - val_accuracy: 0.0000e+00
Epoch 4/50
17/18 [===========================>..] - ETA: 0s - loss: 0.6699 - accuracy: 0.6121
Epoch 4: val_loss did not improve from 0.

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("../dataset", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: test/me/me_37.mp4
1/1 [==============================] - 0s 23ms/step
  me: 86.43%
  baby: 13.57%


In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)


Test video path: test/me/me_85.mp4
1/1 [==============================] - 0s 32ms/step
  me: 54.18%
  baby: 45.82%
